# Import Library

In [1]:
import yfinance as yf
import pandas as pd

# Data Scraping

In [2]:
# List ticker untuk saham 10 perusahaan Indonesia
tickers = ['BBCA.JK', 'BYAN.JK', 'TPIA.JK', 'BBRI.JK', 'BMRI.JK', 'DSSA.JK', 'TLKM.JK', 'ASII.JK', 'BBNI.JK', 'ICBP.JK']

# Download data harga penutupan (adjusted close) untuk setiap perusahaan dengan interval 1 hari
stock_datas = {}
for ticker in tickers:
  df = yf.download(ticker, start='2018-01-01', end='2025-05-05', interval='1d')
  stock_datas[ticker] = df['Close']

# Gabung semua jadi 1 dataframe
df_all = pd.concat(stock_datas, axis=1)
df_all.columns = tickers

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [11]:
df_all.columns = df_all.columns.str.replace('.JK', '', regex=False)

In [12]:
df_all

,BBCA,BYAN,TPIA,BBRI,BMRI,DSSA,TLKM,ASII,BBNI,ICBP
Date,,,,,,,,,,
2018-01-01,3723.926514,711.824585,1457.917480,2289.992676,2592.610352,1390.0,3383.593994,5748.219727,3732.603516,7525.144531
2018-01-02,3723.926514,718.539856,1439.693726,2283.701172,2543.998779,1410.0,3360.732178,5678.964355,3685.474365,7694.247070
2018-01-03,3723.926514,718.539856,1403.245728,2271.119141,2495.387451,1390.0,3223.559326,5575.079102,3553.514648,7694.247070
2018-01-04,3779.190674,711.824585,1348.573608,2233.371826,2535.896729,1390.0,3215.938721,5696.277344,3544.088623,7715.386719
2018-01-05,3783.440918,715.182190,1385.021484,2258.536621,2535.896729,1390.0,3261.663086,5748.219727,3506.385742,7842.214844
...,...,...,...,...,...,...,...,...,...,...
2025-04-25,8600.000000,19950.000000,7800.000000,3740.000000,4900.000000,42250.0,2600.000000,4860.000000,4200.000000,11000.000000
2025-04-28,8775.000000,19975.000000,7800.000000,3840.000000,4920.000000,44850.0,2560.000000,4890.000000,4190.000000,11000.000000
2025-04-29,8725.000000,19875.000000,7675.000000,3850.000000,4940.000000,45875.0,2590.000000,4800.000000,4140.000000,11250.000000


In [13]:
# Saving data to csv
df_all.to_csv("stocks.csv", index=False)

# Data Exploration

In [14]:
df_all.isnull().sum()

BBCA    0
BYAN    0
TPIA    0
BBRI    0
BMRI    0
DSSA    0
TLKM    1
ASII    1
BBNI    0
ICBP    1
dtype: int64

In [15]:
# Melihat baris yang bernilai null
df_all[df_all.isnull().any(axis=1)]

,BBCA,BYAN,TPIA,BBRI,BMRI,DSSA,TLKM,ASII,BBNI,ICBP
Date,,,,,,,,,,
2019-06-19,5152.053711,1144.98999,1209.519653,2880.276123,2881.316895,1450.0,NaN,NaN,3534.166748,NaN
